# Ejercicio Práctico: Sistema de Compra de Billetes de Tren

En este ejercicio vamos a crear una aplicación de terminal que:

1. Permite al usuario **comprar un billete de tren**
2. Genera un **fichero JSON** con los datos del billete
3. Registra la venta en un **fichero CSV** (histórico de ventas)

---

## Estructura del programa

```
┌─────────────────────────────────────┐
│         COMPRA DE BILLETE           │
├─────────────────────────────────────┤
│  1. Usuario elige ruta              │
│  2. Usuario introduce sus datos     │
│  3. Se genera billete (JSON)        │
│  4. Se registra venta (CSV)         │
│  5. Se muestra resumen              │
└─────────────────────────────────────┘
```

### Ficheros que se generan:

- `billete_XXXXX.json` → Billete individual del pasajero
- `ventas.csv` → Histórico de todas las ventas

---

## Código del programa

In [1]:
import json
import csv
import random
from datetime import datetime

# ============================================
# DATOS PREDEFINIDOS
# ============================================

RUTAS = {
    1: {"origen": "Madrid", "destino": "Barcelona", "precio": 45.50, "duracion": "2h 30min"},
    2: {"origen": "Madrid", "destino": "Sevilla", "precio": 38.00, "duracion": "2h 20min"},
    3: {"origen": "Barcelona", "destino": "Valencia", "precio": 25.00, "duracion": "1h 40min"},
    4: {"origen": "Sevilla", "destino": "Málaga", "precio": 22.50, "duracion": "1h 50min"},
    5: {"origen": "Valencia", "destino": "Madrid", "precio": 35.00, "duracion": "1h 45min"}
}

FICHERO_VENTAS = "ventas.csv"

In [2]:
# ============================================
# FUNCIONES
# ============================================

def mostrar_rutas() -> None:
    """Muestra las rutas disponibles"""
    print("\n" + "=" * 50)
    print("           TRENES DISPONIBLES")
    print("=" * 50)
    for num, ruta in RUTAS.items():
        print(f"  {num}. {ruta['origen']} → {ruta['destino']}")
        print(f"     Precio: {ruta['precio']:.2f}€ | Duración: {ruta['duracion']}")
        print()
    print("=" * 50)


def pedir_ruta() -> dict:
    """Pide al usuario que elija una ruta
    
    Returns:
        Diccionario con los datos de la ruta elegida
    """
    mostrar_rutas()
    
    opcion = 0
    while opcion < 1 or opcion > len(RUTAS):
        opcion_str = input("Elige una ruta (1-5): ")
        opcion = int(opcion_str)
        if opcion < 1 or opcion > len(RUTAS):
            print("Opción no válida. Intenta de nuevo.")
    
    return RUTAS[opcion]


def pedir_datos_pasajero() -> dict:
    """Pide los datos del pasajero
    
    Returns:
        Diccionario con nombre y DNI del pasajero
    """
    print("\n--- Datos del pasajero ---")
    nombre = input("Nombre completo: ")
    dni = input("DNI: ")
    
    return {"nombre": nombre, "dni": dni}


def generar_codigo_billete() -> str:
    """Genera un código único para el billete
    
    Returns:
        Código de 5 dígitos aleatorios
    """
    return str(random.randint(10000, 99999))


def crear_billete(ruta: dict, pasajero: dict) -> dict:
    """Crea el billete con todos los datos
    
    Args:
        ruta: Diccionario con origen, destino, precio, duracion
        pasajero: Diccionario con nombre y dni
        
    Returns:
        Diccionario con todos los datos del billete
    """
    codigo = generar_codigo_billete()
    fecha_compra = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    
    billete = {
        "codigo": codigo,
        "pasajero": pasajero,
        "viaje": {
            "origen": ruta["origen"],
            "destino": ruta["destino"],
            "duracion": ruta["duracion"]
        },
        "precio": ruta["precio"],
        "fecha_compra": fecha_compra
    }
    
    return billete


def guardar_billete_json(billete: dict) -> str:
    """Guarda el billete en un fichero JSON
    
    Args:
        billete: Diccionario con los datos del billete
        
    Returns:
        Nombre del fichero creado
    """
    nombre_fichero = f"billete_{billete['codigo']}.json"
    
    with open(nombre_fichero, "w") as fichero:
        json.dump(billete, fichero, indent=4)
    
    return nombre_fichero


def registrar_venta_csv(billete: dict) -> None:
    """Registra la venta en el fichero CSV de histórico
    
    Args:
        billete: Diccionario con los datos del billete
    """
    # Comprobar si el fichero existe para escribir cabecera
    fichero_existe = False
    try:
        with open(FICHERO_VENTAS, "r") as f:
            fichero_existe = True
    except FileNotFoundError:
        fichero_existe = False
    
    # Abrir en modo append para añadir al final
    with open(FICHERO_VENTAS, "a", newline="") as fichero:
        campos = ["codigo", "nombre", "dni", "origen", "destino", "precio", "fecha_compra"]
        escritor = csv.DictWriter(fichero, fieldnames=campos)
        
        # Si el fichero no existía, escribimos la cabecera
        if not fichero_existe:
            escritor.writeheader()
        
        # Escribir la fila de esta venta
        fila = {
            "codigo": billete["codigo"],
            "nombre": billete["pasajero"]["nombre"],
            "dni": billete["pasajero"]["dni"],
            "origen": billete["viaje"]["origen"],
            "destino": billete["viaje"]["destino"],
            "precio": billete["precio"],
            "fecha_compra": billete["fecha_compra"]
        }
        escritor.writerow(fila)


def mostrar_resumen(billete: dict, fichero_json: str) -> None:
    """Muestra el resumen de la compra
    
    Args:
        billete: Diccionario con los datos del billete
        fichero_json: Nombre del fichero JSON generado
    """
    print("\n" + "=" * 50)
    print("         ¡COMPRA REALIZADA CON ÉXITO!")
    print("=" * 50)
    print(f"  Código de billete: {billete['codigo']}")
    print(f"  Pasajero: {billete['pasajero']['nombre']}")
    print(f"  DNI: {billete['pasajero']['dni']}")
    print(f"  Trayecto: {billete['viaje']['origen']} → {billete['viaje']['destino']}")
    print(f"  Duración: {billete['viaje']['duracion']}")
    print(f"  Precio: {billete['precio']:.2f}€")
    print(f"  Fecha: {billete['fecha_compra']}")
    print("-" * 50)
    print(f"  📄 Billete guardado en: {fichero_json}")
    print(f"  📊 Venta registrada en: {FICHERO_VENTAS}")
    print("=" * 50)

In [3]:
# ============================================
# PROGRAMA PRINCIPAL
# ============================================

print("\n🚂 BIENVENIDO AL SISTEMA DE COMPRA DE BILLETES 🚂")

# 1. Elegir ruta
ruta = pedir_ruta()

# 2. Pedir datos del pasajero
pasajero = pedir_datos_pasajero()

# 3. Crear el billete
billete = crear_billete(ruta, pasajero)

# 4. Guardar billete en JSON
fichero_json = guardar_billete_json(billete)

# 5. Registrar venta en CSV
registrar_venta_csv(billete)

# 6. Mostrar resumen
mostrar_resumen(billete, fichero_json)


🚂 BIENVENIDO AL SISTEMA DE COMPRA DE BILLETES 🚂

           TRENES DISPONIBLES
  1. Madrid → Barcelona
     Precio: 45.50€ | Duración: 2h 30min

  2. Madrid → Sevilla
     Precio: 38.00€ | Duración: 2h 20min

  3. Barcelona → Valencia
     Precio: 25.00€ | Duración: 1h 40min

  4. Sevilla → Málaga
     Precio: 22.50€ | Duración: 1h 50min

  5. Valencia → Madrid
     Precio: 35.00€ | Duración: 1h 45min


--- Datos del pasajero ---

         ¡COMPRA REALIZADA CON ÉXITO!
  Código de billete: 58577
  Pasajero: Arancha
  DNI: 06033087a
  Trayecto: Madrid → Barcelona
  Duración: 2h 30min
  Precio: 45.50€
  Fecha: 2026-01-28 15:58:49
--------------------------------------------------
  📄 Billete guardado en: billete_58577.json
  📊 Venta registrada en: ventas.csv


---

## Verificar los ficheros generados

### Ver el billete JSON

In [ ]:
# Ver el contenido del último billete generado
import os

# Buscar ficheros de billete
billetes = [f for f in os.listdir() if f.startswith("billete_") and f.endswith(".json")]

if billetes:
    ultimo_billete = sorted(billetes)[-1]
    print(f"Contenido de {ultimo_billete}:")
    print("-" * 40)
    with open(ultimo_billete, "r") as f:
        print(f.read())
else:
    print("No hay billetes generados")

### Ver el histórico de ventas (CSV)

In [ ]:
# Ver el contenido del CSV
print("Contenido de ventas.csv:")
print("-" * 40)

try:
    with open(FICHERO_VENTAS, "r") as f:
        print(f.read())
except FileNotFoundError:
    print("Aún no hay ventas registradas")

In [ ]:
# Ver el CSV de forma más bonita
import csv

print("\n📊 HISTÓRICO DE VENTAS")
print("=" * 80)

try:
    with open(FICHERO_VENTAS, "r") as fichero:
        lector = csv.DictReader(fichero)
        for fila in lector:
            print(f"[{fila['codigo']}] {fila['nombre']} | {fila['origen']}→{fila['destino']} | {fila['precio']}€")
except FileNotFoundError:
    print("Aún no hay ventas registradas")

---

## 📝 Resumen de lo aprendido

En este ejercicio hemos practicado:

| Concepto | Uso en el ejercicio |
|----------|--------------------|
| **JSON - escritura** | Guardar el billete con `json.dump()` |
| **JSON - estructura** | Diccionarios anidados (pasajero, viaje) |
| **CSV - escritura** | Registrar ventas con `csv.DictWriter()` |
| **CSV - append** | Añadir filas sin borrar las anteriores |
| **CSV - cabecera** | Escribir cabecera solo si el fichero no existe |
| **Funciones** | Modularización del código |
| **Gestión de errores** | `try/except` para comprobar si existe el fichero |